In [1]:
import os
import csv
import time
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as kb
from tensorflow import keras
from tensorflow.keras.layers import RNN, LSTM, TimeDistributed, LSTMCell, RNN, Dense
from tensorflow.keras import Model

import matplotlib.pyplot as plt
from scipy import interpolate
from itertools import combinations
from scipy.signal import find_peaks, butter, filtfilt, lfilter

In [2]:
def peakDetector(array, threshold):
    peaks = []
    idx_start = 0
    idx_end = 2
    
    while(idx_end < len(array)):
        tmp = array[idx_start:idx_end+1]
        max_ = np.max(tmp)
        maxidx_ = np.argmax(tmp)
        
        if maxidx_ == 0:
            idx_start += 1
            idx_end = idx_start + 2
            
        else:
            min1 = np.min(tmp[0:maxidx_])
            min2 = np.min(tmp[maxidx_:len(tmp)])
            
            if min1 + threshold < max_ and min2 + threshold < max_:
                peaks.append(maxidx_ + idx_start)
                idx_start = idx_end + 1
                idx_end = idx_start + 2
            else:
                idx_end += 1
        
    if len(peaks) > 0:
        return True, peaks
    else:
        return False, peaks

In [3]:
def butter_lowpass_filter_3d(data, cutoff, fs, order):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    x = filtfilt(b, a, data[:, 0])
    y = filtfilt(b, a, data[:, 1])
    z = filtfilt(b, a, data[:, 2])
    result = np.stack((x, y), axis=1)
    result = np.append(result, np.expand_dims(z, axis=1), axis=1)
    return result

def getKneeAngle(a, b, c):
    b2a = a - b
    b2c = c - b
    b2a_norm = np.linalg.norm(b2a, axis = 1)
    b2c_norm = np.linalg.norm(b2c, axis = 1)
    inner = np.diag(np.inner(b2a, b2c))
    angle = 180 - np.arccos(inner/(b2a_norm * b2c_norm)) * 180 / np.pi
    return angle

In [5]:
def getHipAngle(a, b, c, d):
    b2a = a - b
    c2d = d - c
    b2a_norm = np.linalg.norm(b2a, axis = 1)
    c2d_norm = np.linalg.norm(c2d, axis = 1)
    inner = np.diag(np.inner(b2a, c2d))
    angle = 180 - np.arccos(inner/(b2a_norm * c2d_norm)) * 180 / np.pi
    return angle

In [6]:
def getZebirsData(file):
    path_l = os.path.join(file, "butterfly_force_curve-L.csv")
    path_r = os.path.join(file, "butterfly_force_curve-R.csv")
    
    csv_data_l = np.loadtxt(path_l, delimiter=',', skiprows=4, usecols=range(2), encoding='utf-8')
    Ltime = csv_data_l[:,0]
    Lforce = csv_data_l[:,1]

    l_contact = []
    l_off = []
    l_contact.append(Ltime[0])
    pre = Ltime[0]
    for i in Ltime:
        if i - pre > 0.2:
            l_contact.append(i)
            if pre != 0:
                l_off.append(pre)
        pre = i
    l_off.append(Ltime[-1])


    csv_data_r = np.loadtxt(path_r, delimiter=',', skiprows=4, usecols=range(2), encoding='utf-8')
    Rtime = csv_data_r[:,0]
    Rforce = csv_data_r[:,1]

    r_contact = []
    r_off = []
    r_contact.append(Rtime[0])
    pre = Rtime[0]
    for i in Rtime:
        if i - pre > 0.2:
            r_contact.append(i)
            if pre != 0:
                r_off.append(pre)
        pre = i
    r_off.append(Rtime[-1])
    
    if r_contact[0] == 0.01:
        return True, l_contact, l_off, r_contact, r_off
    else:
        return False, l_contact, l_off, r_contact, r_off
    
    

In [7]:
def getAzureData(file):
    csv_data = np.loadtxt(file, delimiter=',', skiprows=2, usecols=(101))

    idx_footSwitch = 0
    for i in csv_data:
        if i==0:
            idx_footSwitch+=1
        else:
            break

    
    csv_data_ = np.loadtxt(file, delimiter=',', dtype=str, skiprows=2, usecols=(1))
    line_num=csv_data_.shape[0]
    for i, v in enumerate(csv_data_):
        if v == " ":
            line_num = i
            break
    csv_data = np.loadtxt(file, delimiter=',', skiprows=2, max_rows=line_num, usecols=range(1, 101))
    
    
    #print(csv_data)
    
    #print(csv_data.shape)
    time = csv_data[:,(96, 97, 98, 99)]
    time_ = time[:, 0] * 3600 + time[:,1] * 60 + time[:, 2]+ time[:, 3] / 1000.0
    time_ = time_ - time_[0]
    initialContactTime = time_[idx_footSwitch]
    
    cutoff = 10

    pelvis_idx = 0
    spine_naval_idx = 1
    spine_chest_idx = 2
    neck_idx = 3
    
    Pelvis = csv_data[:,pelvis_idx * 3:(pelvis_idx + 1) * 3]
    Spine_naval = csv_data[:,spine_naval_idx * 3:(spine_naval_idx + 1) * 3]
    Spine_chest = csv_data[:,spine_chest_idx * 3:(spine_chest_idx + 1) * 3]
    Neck = csv_data[:,neck_idx * 3:(neck_idx + 1) * 3]
    
    Pelvis_filtered = butter_lowpass_filter_3d(Pelvis, cutoff, 60, 3)
    Spine_naval_filtered = butter_lowpass_filter_3d(Spine_naval, cutoff, 60, 3)
    Spine_chest_filtered = butter_lowpass_filter_3d(Spine_chest, cutoff, 60, 3)
    Neck_filtered = butter_lowpass_filter_3d(Neck, cutoff, 60, 3)
    
    Angle_chest_filtered = getKneeAngle(Neck_filtered, Spine_chest_filtered, Spine_naval_filtered)
    Angle_naval_filtered = getKneeAngle(Spine_chest_filtered, Spine_naval_filtered, Pelvis_filtered)
    
    Angle_chest = getKneeAngle(Neck, Spine_chest, Spine_naval)
    Angle_naval = getKneeAngle(Spine_chest, Spine_naval, Pelvis)
    
    safe_start=0
    safe_end=0
    
    
    for i, v in enumerate(Pelvis[:, 2]):
        if safe_start == 0 and v < 3300:
            safe_start = i
        if safe_start != 0 and v < 1100:
            safe_end = i
            break
    
    LHip_idx = 18 
    LKnee_idx = 19
    LAnkle_idx = 20
    RHip_idx = 22
    RKnee_idx = 23
    RAnkle_idx = 24

    LHip = csv_data[:, LHip_idx * 3: (LHip_idx + 1) * 3]
    LKnee = csv_data[:, LKnee_idx * 3: (LKnee_idx + 1) * 3]
    LAnkle = csv_data[:, LAnkle_idx * 3: (LAnkle_idx + 1) * 3]
    LHip_filtered = butter_lowpass_filter_3d(LHip, cutoff, 60, 3)
    LKnee_filtered = butter_lowpass_filter_3d(LKnee, cutoff, 60, 3)
    LAnkle_filtered = butter_lowpass_filter_3d(LAnkle, cutoff, 60, 3)
    LAngle_knee_filtered = getKneeAngle(LHip_filtered, LKnee_filtered, LAnkle_filtered)
    LAngle_hip_filtered = getHipAngle(Spine_chest_filtered, Pelvis_filtered, LHip_filtered, LKnee_filtered)
    LAngle_knee = getKneeAngle(LHip, LKnee, LAnkle)
    LAngle_hip = getHipAngle(Spine_chest, Pelvis, LHip, LKnee)


    RHip = csv_data[:, RHip_idx * 3: (RHip_idx + 1) * 3]
    RKnee = csv_data[:, RKnee_idx * 3: (RKnee_idx + 1) * 3]
    RAnkle = csv_data[:, RAnkle_idx * 3: (RAnkle_idx + 1) * 3]
    RHip_filtered = butter_lowpass_filter_3d(RHip, cutoff, 60, 3)
    RKnee_filtered = butter_lowpass_filter_3d(RKnee, cutoff, 60, 3)
    RAnkle_filtered = butter_lowpass_filter_3d(RAnkle, cutoff, 60, 3)
    RAngle_knee_filtered = getKneeAngle(RHip_filtered, RKnee_filtered, RAnkle_filtered)
    RAngle_hip_filtered = getHipAngle(Spine_chest_filtered, Pelvis_filtered, RHip_filtered, RKnee_filtered)
    RAngle_knee = getKneeAngle(RHip, RKnee, RAnkle)
    RAngle_hip = getHipAngle(Spine_chest, Pelvis, RHip, RKnee)
    
    
    return Angle_chest_filtered, Angle_naval_filtered, LAngle_hip_filtered, LAngle_knee_filtered, RAngle_hip_filtered, RAngle_knee_filtered, time_, initialContactTime
    

In [8]:

result = []

class EarlyStoppingAtMinLoss(keras.callbacks.Callback):
    global path_dataset
    global result
    def __init__(self, patience=10):
        super(EarlyStoppingAtMinLoss, self).__init__()
        self.patience = patience
        self.best_weights = None
        
    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        self.best = np.Inf
        
    def on_epoch_end(self, epoch, logs=None):
        current = logs.get("loss")
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait +=1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                self.model.set_weights(self.best_weights)
                
    def on_train_end(self, logs=None):
        global result
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" %(self.stopped_epoch + 1))
        
        coverage = getCoverage(path_dataset, self.model.train_names, self.model)
        result.append(coverage)
        print(self.model.train_names)
        print(coverage)
        
        tmp = list(combinations(names, 6))
        idx_ = tmp.index(self.model.train_names)
        ratio = self.model.ratio
        if idx_ == len(tmp)-1:
            
            for i in result:
                f = open('result.txt', 'a')
                f.write(str(i) + '\n')
                f.close()
                
            print(result)

            if ratio-5 < 20:
                return
            else:
                f = open('result.txt', 'a')
                f.write('ratio : ' + str(ratio - 5) + '\n')
                f.close()
                result = []
                startNewSession(ratio-5, tmp[0])
                
        else:
            startNewSession(ratio, tmp[idx_+1])

In [9]:
# path_dataset = "Dataset"
path_dataset ="dataset"
startRatio = 45

def startNewSession(ratio, train_names):
    epoch = 200
    model = RNN(ratio, train_names)
    model.compile(optimizer=keras.optimizers.Adam(0.001),loss=custom_loss(ratio),metrics=['accuracy'])
    model.fit(model.dataset, verbose=0, epochs=epoch, callbacks=[EarlyStoppingAtMinLoss()])
    
    
def custom_loss(ratio_):
    def loss(y_actual,y_pred):
        y_ = kb.sigmoid(y_pred)
        loss_ = ratio_ * y_actual * kb.log(y_) + (1- y_actual) * kb.log(1 - y_)
        return -tf.reduce_mean(loss_)
    return loss

class RNN(Model):
    global path_dataset
    def __init__(self, ratio_, train_names_):
        super().__init__()
        self.ratio = ratio_
        self.train_names = train_names_
        self.lstm1 = LSTM(15, return_sequences=True)
        self.lstm2 = LSTM(10, return_sequences=True)
        self.lastLayer = TimeDistributed(Dense(1, activation = 'sigmoid'))
        
        self.dataset = getDataset(path_dataset, train_names)

    def call(self, inputs):
        x = self.lstm1(inputs)
        x = self.lstm2(x)
        x = self.lastLayer(x)
        print('rnn', x.shape)
        return x
    
names = next(os.walk(path_dataset))[1]
train_names = list(combinations(names, 6))[0]

f = open('result.txt', 'a')
f.write('ratio : ' + str(startRatio) + '\n')
f.close()

startNewSession(startRatio, train_names)
                
                
                
#startNewSession(50, train_names)
    
"""
                while(l_off[-1] > time[-1]):
                    l_off = l_off[:-1]

                while(r_off[-1] > time[-1]):
                    r_off = r_off[:-1]


                fig, ax = plt.subplots(2, figsize=(18,12))
                ax[0].plot(time, LKnee, 'r-', label = 'LKnee')
                ax[0].plot(time, LHip, 'b-', label = 'LHip')
                ax[0].axvline(x = initialContactTime, label='foot switch on ! at x = {}'.format(initialContactTime), c='b')
                for i in l_contact:
                    ax[0].axvline(x=i, label='contact ! at x = {}'.format(i), c='k')
                for i in l_off:
                    ax[0].axvline(x=i, label='off ! at x = {}'.format(i), c='k', ls = '--')
                ax[0].axis((time[0], time[-1], 0, 90))


                ax[1].plot(time, RKnee, 'r-', label = 'RKnee')
                ax[1].plot(time, RHip, 'b-', label = 'RHip')
                ax[1].axvline(x = initialContactTime, label='foot switch on ! at x = {}'.format(initialContactTime), c='b')
                for i in r_contact:
                    ax[1].axvline(x=i, label='contact ! at x = {}'.format(i), c='k')
                for i in r_off:
                    ax[1].axvline(x=i, label='off ! at x = {}'.format(i), c='k', ls = '--')
                ax[1].axis((time[0], time[-1], 0, 90))

                fig.tight_layout()
                print('2020_11_06_synced//' + name + path_zebris[idx] + '.png')
                plt.savefig('2020_11_06_synced//' + name + path_zebris[idx] + '.png')
                plt.close()
                
                
                
                
                
                
                
                                fig, ax = plt.subplots(2, figsize=(18,12))
                ax[0].plot(time, LKnee, 'r-', label = 'LKnee')
                ax[0].plot(time, LHip, 'b-', label = 'LHip')
                ax[0].axvline(x = initialContactTime, label='foot switch on ! at x = {}'.format(initialContactTime), c='b')
                for i in l_contact:
                    ax[0].axvline(x=i, label='contact ! at x = {}'.format(i), c='k')
                ax[0].axvline(x=time[safe_start], label='safe_start at x = {}'.format(i), c='g')
                ax[0].axvline(x=time[safe_end], label='safe_end at x = {}'.format(i), c='g')
                ax[0].axis((time[0], time[-1], 0, 90))

                # INPUT
                x_new = np.linspace(time[0], time[-1], len(time))

                f = interpolate.interp1d(time, Chest)
                Chest_ = f(x_new)
                f = interpolate.interp1d(time, Naval)
                Naval_ = f(x_new)
                f = interpolate.interp1d(time, LKnee)
                LKnee_ = f(x_new)
                f = interpolate.interp1d(time, LHip)
                LHip_ = f(x_new)
                f = interpolate.interp1d(time, RKnee)
                RKnee_ = f(x_new)
                f = interpolate.interp1d(time, RHip)
                RHip_ = f(x_new)

                RLinput = np.concatenate((np.expand_dims(Chest_, axis=1), np.expand_dims(Naval_, axis=1), np.expand_dims(LKnee_, axis=1), np.expand_dims(LHip_, axis=1), np.expand_dims(RKnee_, axis=1), np.expand_dims(RHip_, axis=1)), axis=1) 
                min_ = np.expand_dims(np.min(RLinput,axis=0), axis=0)
                max_ = np.expand_dims(np.max(RLinput,axis=0), axis=0)
                RLinput = (RLinput - min_) / (max_ - min_)

                # OUTPUT
                Lcontact_output = np.zeros(len(Chest))
                for i in l_contact:
                    tmp = abs(time - i)
                    Lcontact_output[np.argmin(tmp)] = 1



                for idx_ in range(safe_start, safe_end - 40):
                    count_total +=1

                    #RLinput = np.concatenate((np.expand_dims(Chest, axis=1), np.expand_dims(Naval, axis=1), np.expand_dims(LKnee, axis=1), np.expand_dims(LHip, axis=1), np.expand_dims(RKnee, axis=1), np.expand_dims(RHip, axis=1)), axis=1) 
                    testinput = np.expand_dims(RLinput[idx_: idx_+40], axis=0)
                    #min_input = np.expand_dims(np.min(testinput,axis=1), axis=0)
                    #max_input = np.expand_dims(np.max(testinput,axis=1), axis=0)
                    #testinput = (testinput - min_input) / (max_input - min_input)
                    print(model.predict(testinput).shape)
                    output_predicted = np.squeeze(model.predict(testinput))
                    answer = Lcontact_output[idx_:idx_+40]

                    flag, peaks = peakDetector(output_predicted, 0.5)
                    x = time[idx_:idx_+40]
                    if flag:
                        if np.count_nonzero(answer == 1) == len(peaks):
                            ax[1].plot(x[peaks], output_predicted[peaks], 'g.')
                            count_positive += 1
                        else:
                            ax[1].plot(x[peaks], output_predicted[peaks] / 2, 'r.')
                    else:
                        if np.count_nonzero(answer == 1) == 0:
                            count_positive += 1
                        else:
                            ax[1].plot(x[peaks], output_predicted[peaks] / 3 * 2, 'k.')



                    #
                    #ax[1].plot(x, output_predicted, 'k.')

                for i in l_contact:
                    ax[1].axvline(x=i, label='contact ! at x = {}'.format(i), c='y')
                ax[1].axvline(x=time[safe_start], label='safe_start at x = {}'.format(i), c='g')
                ax[1].axvline(x=time[safe_end], label='safe_end at x = {}'.format(i), c='g')
                ax[1].axis((time[0], time[-1], 0, 1.2))


                fig.tight_layout()
                #print('2020_11_06_testResult//' + name + path_zebris[idx] + '.png')
                #plt.savefig('2020_11_06_testResult//' + name + path_zebris[idx] + '.png')
                plt.close()

"""

StopIteration: 

In [8]:
path_dataset = "Dataset"
def getDataset(path_dataset, train_names):
    totalDataset = None
    numFrame = 40
    for name in next(os.walk(path_dataset))[1]:
        if name in train_names:
            #print(name)
            path_person = os.path.join(path_dataset, name)
            (root, path_zebris, path_azure) = next(os.walk(path_person))
            if '.ipynb_checkpoints' in path_zebris:
                path_zebris.remove('.ipynb_checkpoints')
            path_zebris = sorted(path_zebris)
            path_azure = sorted(path_azure)
            
            for idx in range(len(path_zebris)):
                #print(path_zebris[idx])
                Chest, Naval, LHip, LKnee, RHip, RKnee, time, initialContactTime = getAzureData(os.path.join(path_person, path_azure[idx]))
                firstStep, l_contact, l_off, r_contact, r_off = getZebirsData(os.path.join(path_person, path_zebris[idx]))

                timeGap = initialContactTime - r_contact[0]
                l_contact = l_contact + timeGap
                l_off = l_off + timeGap
                r_contact = r_contact + timeGap
                r_off = r_off + timeGap
                
                # INPUT
                x_new = np.linspace(time[0], time[-1], len(time))

                f = interpolate.interp1d(time, Chest)
                Chest_ = f(x_new)
                f = interpolate.interp1d(time, Naval)
                Naval_ = f(x_new)
                f = interpolate.interp1d(time, LKnee)
                LKnee_ = f(x_new)
                f = interpolate.interp1d(time, LHip)
                LHip_ = f(x_new)
                f = interpolate.interp1d(time, RKnee)
                RKnee_ = f(x_new)
                f = interpolate.interp1d(time, RHip)
                RHip_ = f(x_new)

                RLinput = np.concatenate((np.expand_dims(Chest_, axis=1), np.expand_dims(Naval_, axis=1), np.expand_dims(LKnee_, axis=1), np.expand_dims(LHip_, axis=1), np.expand_dims(RKnee_, axis=1), np.expand_dims(RHip_, axis=1)), axis=1) 
                min_ = np.expand_dims(np.min(RLinput,axis=0), axis=0)
                max_ = np.expand_dims(np.max(RLinput,axis=0), axis=0)
                RLinput = (RLinput - min_) / (max_ - min_)

                # OUTPUT
                Lcontact_output = np.zeros(len(Chest))
                for i in l_contact:
                    tmp = abs(time - i)
                    Lcontact_output[np.argmin(tmp)] = 1

                # Slicing
                inputs =  np.empty((1, numFrame, RLinput.shape[1]), dtype=float)
                outputs = np.empty((1, numFrame), dtype=float)

                tmp = abs(time - r_contact[0])
                safe_start = np.argmin(tmp)

                tmp = abs(time - max(l_off[-1], r_off[-1]))
                safe_end = np.argmin(tmp)


                idx_end = numFrame


                while(idx_end <safe_end):
                    if(idx_end - numFrame > safe_start):
                        input_ = np.expand_dims(RLinput[idx_end - numFrame: idx_end], axis=0)
                        output_ = np.expand_dims(Lcontact_output[idx_end - numFrame:idx_end], axis=0)

                        inputs = np.append(inputs, input_, axis=0)
                        outputs = np.append(outputs, output_, axis=0)
                    idx_end +=1

                inputs = np.delete(inputs, [0,0,0], axis=0)
                outputs = np.delete(outputs, [0,0], axis=0)

                dataset = tf.data.Dataset.from_tensor_slices((inputs.astype('float32'), outputs.astype('float32')))
                if totalDataset == None:
                    totalDataset = dataset
                else:
                    totalDataset = totalDataset.concatenate(dataset)
                    #dataset_shuffled = dataset.shuffle(10000).batch(32, drop_remainder = True)
                    
    return totalDataset.shuffle(10000).batch(32, drop_remainder = True)
                    

            

In [9]:
path_dataset = "Dataset"
def getCoverage(path_dataset, train_names, model):
    
    count_total=0
    count_positive=0

    for name in next(os.walk(path_dataset))[1]:
        if name not in train_names:
            #print(name)
            path_person = os.path.join(path_dataset, name)
            (root, path_zebris, path_azure) = next(os.walk(path_person))
            if '.ipynb_checkpoints' in path_zebris:
                path_zebris.remove('.ipynb_checkpoints')
            path_zebris = sorted(path_zebris)
            path_azure = sorted(path_azure)
            for idx in range(len(path_zebris)):
                #print(path_zebris[idx])
                
                Chest, Naval, LHip, LKnee, RHip, RKnee, time, initialContactTime = getAzureData(os.path.join(path_person, path_azure[idx]))
                firstStep, l_contact, l_off, r_contact, r_off = getZebirsData(os.path.join(path_person, path_zebris[idx]))

                timeGap = initialContactTime - r_contact[0]
                l_contact = l_contact + timeGap
                l_off = l_off + timeGap
                r_contact = r_contact + timeGap
                r_off = r_off + timeGap

                """
                while((len(l_off) > 0) and (l_off[-1] > time[-1])):
                    l_off = l_off[:-1]

                while((len(r_off) > 0) and (l_off[-1] > time[-1])):
                    r_off = r_off[:-1]

                """
                
                tmp = abs(time - r_contact[0])
                safe_start = np.argmin(tmp)

                tmp = abs(time - max(l_off[-1], r_off[-1]))
                safe_end = np.argmin(tmp)
                
                # INPUT
                x_new = np.linspace(time[0], time[-1], len(time))

                f = interpolate.interp1d(time, Chest)
                Chest_ = f(x_new)
                f = interpolate.interp1d(time, Naval)
                Naval_ = f(x_new)
                f = interpolate.interp1d(time, LKnee)
                LKnee_ = f(x_new)
                f = interpolate.interp1d(time, LHip)
                LHip_ = f(x_new)
                f = interpolate.interp1d(time, RKnee)
                RKnee_ = f(x_new)
                f = interpolate.interp1d(time, RHip)
                RHip_ = f(x_new)

                RLinput = np.concatenate((np.expand_dims(Chest_, axis=1), np.expand_dims(Naval_, axis=1), np.expand_dims(LKnee_, axis=1), np.expand_dims(LHip_, axis=1), np.expand_dims(RKnee_, axis=1), np.expand_dims(RHip_, axis=1)), axis=1) 
                min_ = np.expand_dims(np.min(RLinput,axis=0), axis=0)
                max_ = np.expand_dims(np.max(RLinput,axis=0), axis=0)
                RLinput = (RLinput - min_) / (max_ - min_)

                # OUTPUT
                Lcontact_output = np.zeros(len(Chest))
                for i in l_contact:
                    tmp = abs(time - i)
                    Lcontact_output[np.argmin(tmp)] = 1



                for idx_ in range(safe_start, safe_end - 40):
                    count_total +=1
                    testinput = np.expand_dims(RLinput[idx_: idx_+40], axis=0)
                    output_predicted = np.squeeze(model.predict(testinput))
                    answer = Lcontact_output[idx_:idx_+40]

                    flag, peaks = peakDetector(output_predicted, 0.5)
                    x = time[idx_:idx_+40]
                    if flag:
                        if np.count_nonzero(answer == 1) == len(peaks):
                            count_positive += 1
                    else:
                        if np.count_nonzero(answer == 1) == 0:
                            count_positive += 1
                            
    return count_positive * 100 / count_total


In [ ]:
path_dataset = "2020_11_06"
for name in next(os.walk(path_dataset))[1]:
    if name in train_names:
        path_person = os.path.join(path_dataset, name)
        (root, path_zebris, path_azure) = next(os.walk(path_person))
        #print(name)
        if '.ipynb_checkpoints' in path_zebris:
            path_zebris.remove('.ipynb_checkpoints')
        path_zebris = sorted(path_zebris)
        path_azure = sorted(path_azure)
        for idx in range(len(path_zebris)):
            Chest, Naval, LHip, LKnee, RHip, RKnee, time, initialContactTime = getAzureData(os.path.join(path_person, path_azure[idx]))
            firstStep, l_contact, l_off, r_contact, r_off = getZebirsData(os.path.join(path_person, path_zebris[idx]))

            timeGap = initialContactTime - r_contact[0]
            l_contact = l_contact + timeGap
            l_off = l_off + timeGap
            r_contact = r_contact + timeGap
            r_off = r_off + timeGap

            while(l_off[-1] > time[-1]):
                l_off = l_off[:-1]

            while(r_off[-1] > time[-1]):
                r_off = r_off[:-1]


            fig, ax = plt.subplots(2, figsize=(18,12))
            ax[0].plot(time, LKnee, 'r-', label = 'LKnee')
            ax[0].plot(time, LHip, 'b-', label = 'LHip')
            ax[0].axvline(x = initialContactTime, label='foot switch on ! at x = {}'.format(initialContactTime), c='b')
            for i in l_contact:
                ax[0].axvline(x=i, label='contact ! at x = {}'.format(i), c='k')
            for i in l_off:
                ax[0].axvline(x=i, label='off ! at x = {}'.format(i), c='k', ls = '--')
            ax[0].axis((time[0], time[-1], 0, 90))


            ax[1].plot(time, RKnee, 'r-', label = 'RKnee')
            ax[1].plot(time, RHip, 'b-', label = 'RHip')
            ax[1].axvline(x = initialContactTime, label='foot switch on ! at x = {}'.format(initialContactTime), c='b')
            for i in r_contact:
                ax[1].axvline(x=i, label='contact ! at x = {}'.format(i), c='k')
            for i in r_off:
                ax[1].axvline(x=i, label='off ! at x = {}'.format(i), c='k', ls = '--')
            ax[1].axis((time[0], time[-1], 0, 90))

            fig.tight_layout()
            print('2020_11_06_synced//' + name + path_zebris[idx] + '.png')
            plt.savefig('2020_11_06_synced//' + name + path_zebris[idx] + '.png')
            plt.close()



            # INPUT
            x_new = np.linspace(time[0], time[-1], len(time))

            f = interpolate.interp1d(time, Chest)
            Chest_ = f(x_new)
            f = interpolate.interp1d(time, Naval)
            Naval_ = f(x_new)
            f = interpolate.interp1d(time, LKnee)
            LKnee_ = f(x_new)
            f = interpolate.interp1d(time, LHip)
            LHip_ = f(x_new)
            f = interpolate.interp1d(time, RKnee)
            RKnee_ = f(x_new)
            f = interpolate.interp1d(time, RHip)
            RHip_ = f(x_new)

            RLinput = np.concatenate((np.expand_dims(Chest_, axis=1), np.expand_dims(Naval_, axis=1), np.expand_dims(LKnee_, axis=1), np.expand_dims(LHip_, axis=1), np.expand_dims(RKnee_, axis=1), np.expand_dims(RHip_, axis=1)), axis=1) 
            min_ = np.expand_dims(np.min(RLinput,axis=0), axis=0)
            max_ = np.expand_dims(np.max(RLinput,axis=0), axis=0)
            RLinput = (RLinput - min_) / (max_ - min_)

            # OUTPUT
            Lcontact_output = np.zeros(len(Chest))
            for i in l_contact:
                tmp = abs(time - i)
                Lcontact_output[np.argmin(tmp)] = 1

            # Slicing
            inputs =  np.empty((1, numFrame, RLinput.shape[1]), dtype=float)
            outputs = np.empty((1, numFrame), dtype=float)

            tmp = abs(time - r_contact[0])
            safe_start = np.argmin(tmp)

            tmp = abs(time - max(l_off[-1], r_off[-1]))
            safe_end = np.argmin(tmp)


            idx_end = numFrame


            while(idx_end <safe_end):
                if(idx_end - numFrame > safe_start):
                    input_ = np.expand_dims(RLinput[idx_end - numFrame: idx_end], axis=0)
                    output_ = np.expand_dims(Lcontact_output[idx_end - numFrame:idx_end], axis=0)

                    inputs = np.append(inputs, input_, axis=0)
                    outputs = np.append(outputs, output_, axis=0)
                idx_end +=1

            inputs = np.delete(inputs, [0,0,0], axis=0)
            outputs = np.delete(outputs, [0,0], axis=0)

            dataset = tf.data.Dataset.from_tensor_slices((inputs.astype('float32'), outputs.astype('float32')))
            if totalDataset == None:
                totalDataset = dataset
            else:
                totalDataset = totalDataset.concatenate(dataset)

                
    
epoch = 100
model = RNN()

dataset_shuffled = totalDataset.shuffle(10000).batch(32, drop_remainder = True)
model.compile(optimizer=keras.optimizers.Adam(0.001),loss=custom_loss,metrics=['accuracy'])
model.fit(dataset_shuffled, epochs=epoch)

#model_name = "Loss_" +str(constant)+ "_Model"
#model.save(model_name)


In [ ]:
#########################

constant = 150

def custom_loss(y_actual,y_pred):
    y_ = kb.sigmoid(y_pred)
    custom_loss = constant * y_actual * kb.log(y_) + (1- y_actual) * kb.log(1 - y_)
    return -tf.reduce_mean(custom_loss)

class RNN(Model):

    def __init__(self):
        super(RNN, self).__init__()
        self.lstm1 = LSTM(15, return_sequences=True)
        self.lstm2 = LSTM(10, return_sequences=True)
        self.lastLayer = TimeDistributed(Dense(1, activation = 'sigmoid'))

    def call(self, inputs):
        x = self.lstm1(inputs)
        x = self.lstm2(x)
        x = self.lastLayer(x)
        print('rnn', x.shape)
        return x
    

path_dataset = "2020_11_06"
names = next(os.walk(path_dataset))[1]

for number in range(1, len(names)):
    print(number)
    coverages = []
    tmp = list(permutations(names, number))
    for train_names in tmp:
        totalDataset = None
        for name in next(os.walk(path_dataset))[1]:
            if name in train_names:
                path_person = os.path.join(path_dataset, name)
                (root, path_zebris, path_azure) = next(os.walk(path_person))
                #print(name)
                if '.ipynb_checkpoints' in path_zebris:
                    path_zebris.remove('.ipynb_checkpoints')
                path_zebris = sorted(path_zebris)
                path_azure = sorted(path_azure)
                for idx in range(len(path_zebris)):
                    Chest, Naval, LHip, LKnee, RHip, RKnee, time, initialContactTime = getAzureData(os.path.join(path_person, path_azure[idx]))
                    firstStep, l_contact, l_off, r_contact, r_off = getZebirsData(os.path.join(path_person, path_zebris[idx]))

                    timeGap = initialContactTime - r_contact[0]
                    l_contact = l_contact + timeGap
                    l_off = l_off + timeGap
                    r_contact = r_contact + timeGap
                    r_off = r_off + timeGap

                    while(l_off[-1] > time[-1]):
                        l_off = l_off[:-1]

                    while(r_off[-1] > time[-1]):
                        r_off = r_off[:-1]
                        
                    # INPUT
                    x_new = np.linspace(time[0], time[-1], len(time))

                    f = interpolate.interp1d(time, Chest)
                    Chest_ = f(x_new)
                    f = interpolate.interp1d(time, Naval)
                    Naval_ = f(x_new)
                    f = interpolate.interp1d(time, LKnee)
                    LKnee_ = f(x_new)
                    f = interpolate.interp1d(time, LHip)
                    LHip_ = f(x_new)
                    f = interpolate.interp1d(time, RKnee)
                    RKnee_ = f(x_new)
                    f = interpolate.interp1d(time, RHip)
                    RHip_ = f(x_new)

                    RLinput = np.concatenate((np.expand_dims(Chest_, axis=1), np.expand_dims(Naval_, axis=1), np.expand_dims(LKnee_, axis=1), np.expand_dims(LHip_, axis=1), np.expand_dims(RKnee_, axis=1), np.expand_dims(RHip_, axis=1)), axis=1) 
                    min_ = np.expand_dims(np.min(RLinput,axis=0), axis=0)
                    max_ = np.expand_dims(np.max(RLinput,axis=0), axis=0)
                    RLinput = (RLinput - min_) / (max_ - min_)

                    # OUTPUT
                    Lcontact_output = np.zeros(len(Chest))
                    for i in l_contact:
                        tmp = abs(time - i)
                        Lcontact_output[np.argmin(tmp)] = 1

                    # Slicing
                    inputs =  np.empty((1, numFrame, RLinput.shape[1]), dtype=float)
                    outputs = np.empty((1, numFrame), dtype=float)

                    tmp = abs(time - r_contact[0])
                    safe_start = np.argmin(tmp)

                    tmp = abs(time - max(l_off[-1], r_off[-1]))
                    safe_end = np.argmin(tmp)


                    idx_end = numFrame


                    while(idx_end <safe_end):
                        if(idx_end - numFrame > safe_start):
                            input_ = np.expand_dims(RLinput[idx_end - numFrame: idx_end], axis=0)
                            output_ = np.expand_dims(Lcontact_output[idx_end - numFrame:idx_end], axis=0)

                            inputs = np.append(inputs, input_, axis=0)
                            outputs = np.append(outputs, output_, axis=0)
                        idx_end +=1

                    inputs = np.delete(inputs, [0,0,0], axis=0)
                    outputs = np.delete(outputs, [0,0], axis=0)

                    dataset = tf.data.Dataset.from_tensor_slices((inputs.astype('float32'), outputs.astype('float32')))
                    if totalDataset == None:
                        totalDataset = dataset
                    else:
                        totalDataset = totalDataset.concatenate(dataset)
        epoch = 100
        model = RNN()
        dataset_shuffled = totalDataset.shuffle(10000).batch(32, drop_remainder = True)
        model.compile(optimizer=keras.optimizers.Adam(0.001),loss=custom_loss,metrics=['accuracy'])
        model.fit(dataset_shuffled, epochs=epoch)
        break
        count_total = 0.0
        count_positive = 0.0
        
        for name in next(os.walk(path_dataset))[1]:
            if name not in train_names:
                path_person = os.path.join(path_dataset, name)
                (root, path_zebris, path_azure) = next(os.walk(path_person))
                if '.ipynb_checkpoints' in path_zebris:
                    path_zebris.remove('.ipynb_checkpoints')
                path_zebris = sorted(path_zebris)
                path_azure = sorted(path_azure)
                for idx in range(len(path_zebris)):
                    Chest, Naval, LHip, LKnee, RHip, RKnee, time, initialContactTime = getAzureData(os.path.join(path_person, path_azure[idx]))
                    firstStep, l_contact, l_off, r_contact, r_off = getZebirsData(os.path.join(path_person, path_zebris[idx]))

                    timeGap = initialContactTime - r_contact[0]
                    l_contact = l_contact + timeGap
                    l_off = l_off + timeGap
                    r_contact = r_contact + timeGap
                    r_off = r_off + timeGap

                    while(l_off[-1] > time[-1]):
                        l_off = l_off[:-1]

                    while(r_off[-1] > time[-1]):
                        r_off = r_off[:-1]


                    tmp = abs(time - r_contact[0])
                    safe_start = np.argmin(tmp)

                    tmp = abs(time - max(l_off[-1], r_off[-1]))
                    safe_end = np.argmin(tmp)

                    # INPUT
                    x_new = np.linspace(time[0], time[-1], len(time))

                    f = interpolate.interp1d(time, Chest)
                    Chest_ = f(x_new)
                    f = interpolate.interp1d(time, Naval)
                    Naval_ = f(x_new)
                    f = interpolate.interp1d(time, LKnee)
                    LKnee_ = f(x_new)
                    f = interpolate.interp1d(time, LHip)
                    LHip_ = f(x_new)
                    f = interpolate.interp1d(time, RKnee)
                    RKnee_ = f(x_new)
                    f = interpolate.interp1d(time, RHip)
                    RHip_ = f(x_new)

                    RLinput = np.concatenate((np.expand_dims(Chest_, axis=1), np.expand_dims(Naval_, axis=1), np.expand_dims(LKnee_, axis=1), np.expand_dims(LHip_, axis=1), np.expand_dims(RKnee_, axis=1), np.expand_dims(RHip_, axis=1)), axis=1) 
                    min_ = np.expand_dims(np.min(RLinput,axis=0), axis=0)
                    max_ = np.expand_dims(np.max(RLinput,axis=0), axis=0)
                    RLinput = (RLinput - min_) / (max_ - min_)

                    # OUTPUT
                    Lcontact_output = np.zeros(len(Chest))
                    for i in l_contact:
                        tmp = abs(time - i)
                        Lcontact_output[np.argmin(tmp)] = 1
                    for idx_ in range(safe_start, safe_end - 40):
                        count_total +=1
                        testinput = np.expand_dims(RLinput[idx_: idx_+40], axis=0)
                        output_predicted = np.squeeze(model.predict(testinput))
                        answer = Lcontact_output[idx_:idx_+40]

                        flag, peaks = peakDetector(output_predicted, 0.5)
                        x = time[idx_:idx_+40]
                        if flag:
                            if np.count_nonzero(answer == 1) == len(peaks):
                                count_positive += 1
                        else:
                            if np.count_nonzero(answer == 1) == 0:
                                count_positive += 1
        coverages.append(count_positive*100 / count_total)
    print('number : {}, Converage_ave : {}, Coverage_std : {}'.format(number, np.mean(coverages), np.std(coverages)))
"""
for name in next(os.walk(path_dataset))[1]:
    if name not in train_names:
        path_person = os.path.join(path_dataset, name)
        (root, path_zebris, path_azure) = next(os.walk(path_person))
        print(name)
        if '.ipynb_checkpoints' in path_zebris:
            path_zebris.remove('.ipynb_checkpoints')
        path_zebris = sorted(path_zebris)
        path_azure = sorted(path_azure)
        for idx in range(len(path_zebris)):
            Chest, Naval, LHip, LKnee, RHip, RKnee, time, initialContactTime = getAzureData(os.path.join(path_person, path_azure[idx]))
            firstStep, l_contact, l_off, r_contact, r_off = getZebirsData(os.path.join(path_person, path_zebris[idx]))

            timeGap = initialContactTime - r_contact[0]
            l_contact = l_contact + timeGap
            l_off = l_off + timeGap
            r_contact = r_contact + timeGap
            r_off = r_off + timeGap
            
            while(l_off[-1] > time[-1]):
                l_off = l_off[:-1]

            while(r_off[-1] > time[-1]):
                r_off = r_off[:-1]
                
                
            tmp = abs(time - r_contact[0])
            safe_start = np.argmin(tmp)

            tmp = abs(time - max(l_off[-1], r_off[-1]))
            safe_end = np.argmin(tmp)



            fig, ax = plt.subplots(2, figsize=(18,12))
            ax[0].plot(time, LKnee, 'r-', label = 'LKnee')
            ax[0].plot(time, LHip, 'b-', label = 'LHip')
            ax[0].axvline(x = initialContactTime, label='foot switch on ! at x = {}'.format(initialContactTime), c='b')
            for i in l_contact:
                ax[0].axvline(x=i, label='contact ! at x = {}'.format(i), c='k')
            ax[0].axvline(x=time[safe_start], label='safe_start at x = {}'.format(i), c='g')
            ax[0].axvline(x=time[safe_end], label='safe_end at x = {}'.format(i), c='g')
            ax[0].axis((time[0], time[-1], 0, 90))
            
            # INPUT
            x_new = np.linspace(time[0], time[-1], len(time))

            f = interpolate.interp1d(time, Chest)
            Chest_ = f(x_new)
            f = interpolate.interp1d(time, Naval)
            Naval_ = f(x_new)
            f = interpolate.interp1d(time, LKnee)
            LKnee_ = f(x_new)
            f = interpolate.interp1d(time, LHip)
            LHip_ = f(x_new)
            f = interpolate.interp1d(time, RKnee)
            RKnee_ = f(x_new)
            f = interpolate.interp1d(time, RHip)
            RHip_ = f(x_new)

            RLinput = np.concatenate((np.expand_dims(Chest_, axis=1), np.expand_dims(Naval_, axis=1), np.expand_dims(LKnee_, axis=1), np.expand_dims(LHip_, axis=1), np.expand_dims(RKnee_, axis=1), np.expand_dims(RHip_, axis=1)), axis=1) 
            min_ = np.expand_dims(np.min(RLinput,axis=0), axis=0)
            max_ = np.expand_dims(np.max(RLinput,axis=0), axis=0)
            RLinput = (RLinput - min_) / (max_ - min_)
            
            # OUTPUT
            Lcontact_output = np.zeros(len(Chest))
            for i in l_contact:
                tmp = abs(time - i)
                Lcontact_output[np.argmin(tmp)] = 1
                
                
            
            for idx_ in range(safe_start, safe_end - 40):
                count_total +=1
                
                #RLinput = np.concatenate((np.expand_dims(Chest, axis=1), np.expand_dims(Naval, axis=1), np.expand_dims(LKnee, axis=1), np.expand_dims(LHip, axis=1), np.expand_dims(RKnee, axis=1), np.expand_dims(RHip, axis=1)), axis=1) 
                testinput = np.expand_dims(RLinput[idx_: idx_+40], axis=0)
                #min_input = np.expand_dims(np.min(testinput,axis=1), axis=0)
                #max_input = np.expand_dims(np.max(testinput,axis=1), axis=0)
                #testinput = (testinput - min_input) / (max_input - min_input)
                output_predicted = np.squeeze(model.predict(testinput))
                answer = Lcontact_output[idx_:idx_+40]
                
                flag, peaks = peakDetector(output_predicted, 0.5)
                x = time[idx_:idx_+40]
                if flag:
                    if np.count_nonzero(answer == 1) == len(peaks):
                        ax[1].plot(x[peaks], output_predicted[peaks], 'g.')
                        count_positive += 1
                    else:
                        ax[1].plot(x[peaks], output_predicted[peaks] / 2, 'r.')
                else:
                    if np.count_nonzero(answer == 1) == 0:
                        count_positive += 1
                    else:
                        ax[1].plot(x[peaks], output_predicted[peaks] / 3 * 2, 'k.')
                        
                

                #
                #ax[1].plot(x, output_predicted, 'k.')

            for i in l_contact:
                ax[1].axvline(x=i, label='contact ! at x = {}'.format(i), c='y')
            ax[1].axvline(x=time[safe_start], label='safe_start at x = {}'.format(i), c='g')
            ax[1].axvline(x=time[safe_end], label='safe_end at x = {}'.format(i), c='g')
            ax[1].axis((time[0], time[-1], 0, 1.2))


            fig.tight_layout()
            #print('2020_11_06_testResult//' + name + path_zebris[idx] + '.png')
            #plt.savefig('2020_11_06_testResult//' + name + path_zebris[idx] + '.png')
            plt.close()
            
print('Loss : {}, Total : {}, Positive : {}, Coverage : {}%'.format(constant, count_total, count_positive, count_positive * 100 / count_total))
"""